## Dataset and Dataloader

引入原因：之前的训练中，我们都是使用所有的数据进行训练的，但是在实际情况下，数据量可能会非常大，一次性加载所有数据到内存中可能会导致内存不足的问题；因此，我们需要使用Dataset and Dataloader来分小的批次(batch)进行训练。

In [11]:
import torch 
import numpy as np
import pandas as pd

from torch.utils.data import Dataset,DataLoader

class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(8,6)
        self.linear2 = torch.nn.Linear(6,4)
        self.linear3 = torch.nn.Linear(4,1)
        self.activate = torch.nn.ReLU()
    
    def forward(self,x):
        x = self.activate(self.linear1(x))
        x = self.activate(self.linear2(x))
        x = self.activate(self.linear3(x))
        return x
model = Model()


class MyDataset(Dataset):
    def __init__(self):
        self.data = pd.read_csv('./data/diabetes.csv',dtype=np.float32)
        self.x_data = torch.from_numpy(self.data.values[:,:-1])
        self.y_data = torch.from_numpy(self.data.values[:,[-1]])
        self.len = self.data.shape[0]
        

    def __getitem__(self, index):
        return self.x_data[index],self.y_data[index]
    def __len__(self):
        return self.len
    
dataset = MyDataset()
dataloader = DataLoader(dataset=dataset,batch_size=32,shuffle=True)

# 损失函数和优化器
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)


In [12]:
# 训练循环

for epoch in range(1000):
    for i,data in enumerate(dataloader,0):
        # 1、准备数据
        inputs,labels = data
        # 2、前向传播
        y_pred = model(inputs)
        # 3、计算损失
        loss = criterion(y_pred,labels)
        # 4、反向传播
        loss.backward()
        # 5、梯度清零
        optimizer.zero_grad()
        # 6、更新参数
        optimizer.step()
    if epoch % 100 == 0:
        print(f'Epoch: {epoch}, loss: {loss.item():.4f}')




RuntimeError: all elements of input should be between 0 and 1

## 2、MNIST 数据集

In [25]:
import torch
from torch.utils.data import Dataset,DataLoader

# datasets是torchvision中提供的数据集,
# transforms是torchvision中提供的变换，可以把图片转换为张量

from torchvision import datasets,transforms

# root | 数据存储的根目录 |
# train | True 训练集 False 测试集 |
# transform | 数据转换 | 数据转换的方式 
# transforms.ToTensor() 的核心作用 转换成张量并归一化
# 它将PIL图像或NumPy数组转换为PyTorch张量，同时进行关键的数值归一化。

train_dataset = datasets.MNIST(root='./data/dataset/MNIST',train=True,transform=transforms.ToTensor(),download=True)
test_dataset = datasets.MNIST(root='./data/dataset/MNIST',train=False,transform=transforms.ToTensor(),download=True)

train_dataloader = DataLoader(dataset=train_dataset,batch_size=32,shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset,batch_size=32,shuffle=True)

# 使用 Subset 获取前 100 个样本
train_dataset_subset = torch.utils.data.Subset(train_dataset, range(100))  # 0:100
test_dataset_subset = torch.utils.data.Subset(test_dataset, range(100))    # 0:100





In [ ]:
from PIL import Image
import torchvision.transforms as transforms

# 原始PIL图像
img_pil = Image.open('./data/dog.jpg')  # 形状: (H, W, C)
print(type(img_pil))  # <class 'PIL.JpegImagePlugin.JpegImageFile'>
print(img_pil.size)   # 例如 (2752, 3440)

# 应用ToTensor转换
transform = transforms.ToTensor()
img_tensor = transform(img_pil)

print(img_tensor.shape)  # torch.Size([C, H, W])，例如 [3, 28, 28]
print(img_tensor.dtype)  # torch.float32
print(img_tensor.min())  # 0.0 (或接近0.0)
print(img_tensor.max())  # 1.0 (或接近1.0)


<class 'PIL.JpegImagePlugin.JpegImageFile'>
(2752, 3440)
torch.Size([3, 3440, 2752])
torch.float32
tensor(0.)
tensor(1.)
